In [1]:
from pynq import DefaultIP

class tensorunit(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        
    bindto = ['EXPOLAR:user:tensorcore:1.0']
    
    def cfg_featsel(self, x, y):
        old = self.read(0x00)
        z1 = (((x<<28)& 0xf000_0000) + (old & 0x0fff_ffff))
        z2 = (((y<<26)& 0x0c00_0000) + (z1 & 0xf3ff_ffff))
        self.write(0x00, z2)
        
    def cfg_bias(self, bias):
        old = self.read(0x00)
        z1 = (((bias<<25)& 0x0200_0000) + (old & 0xfdff_ffff))
        self.write(0x00, z1)
        
    def cfg_wgtsel(self, wgt):
        old = self.read(0x00)
        z1 = (((wgt<<1)& 0x0000_0ffe) + (old & 0xffff_f001))
        self.write(0x00, z1)


    def weight(self, file):
        file_name = file
        base_addr = 256
        rel_addr = 32
        width = 4
        addr_feature_gen_bin(base_addr, rel_addr, width, file_name)
        list_addr, list_feature = addr_feature_gen_bin(base_addr, rel_addr, width, file_name)
        for i in range(len(list_addr)):
            tensor_stat.write(list_addr[i], list_feature[i])

    def feature(self, file):
        file_name = file
        base_addr = 4
        rel_addr = 8
        width = 4
        addr_feature_gen_hex(base_addr, rel_addr, width, file_name)
        list_addr, list_feature = addr_feature_gen_hex(base_addr, rel_addr, width, file_name)
        for i in range(len(list_addr)):
            tensor_stat.write(list_addr[i], list_feature[i])

    def bias(self, file):
        file_name = file
        base_addr = 96
        rel_addr = 8
        width = 4
        addr_feature_gen_hex(base_addr, rel_addr, width, file_name)
        list_addr, list_feature = addr_feature_gen_hex(base_addr, rel_addr, width, file_name)
        for i in range(len(list_addr)):
            tensor_stat.write(list_addr[i], list_feature[i])
            
    def conv3x3(self, width):
        txtName = 'output.txt'
        f = open(txtName, 'w+')
        for wgt in range(0,width):
            overlay.tensorcore_0.cfg_wgtsel(wgt)
            psum0 = tensor_stat.read(0x80)
            psum1 = tensor_stat.read(0x84)
            out_str = '%x'%psum0 + '%x'%psum1 + '\r\n'
            f.write(out_str)



def str2bin(x):
    res = 0
    for i in range(len(x)):
        res = res * 2 + int(x[i])
    return res


def str2hex(x):
    res = 0
    for i in range(len(x)):
        if x[i] >= 'a' and x[i] <= 'z':
            res = res*16 + ord(x[i])-87
        else:
            res = res * 16 + int(x[i])
    return res


def addr_feature_gen_bin(base_addr, rel_addr, width, file_name):
    list_addr = []
    list_feature = []
    fp = open(file_name, "r")
    r_str = fp.readline()
    while r_str != "":
        varlist = r_str.split()
        list_addr.append(base_addr)
        list_addr.append(base_addr + width)
        list_feature.append(str2bin(varlist[1]))
        list_feature.append(str2bin(varlist[0]))
        base_addr = base_addr + rel_addr
        r_str = fp.readline()
    fp.close()

    return list_addr, list_feature	

def addr_feature_gen_hex(base_addr, rel_addr, width, file_name):
    list_addr = []
    list_feature = []
    fp = open(file_name, "r")
    r_str = fp.readline()
    while r_str != "":
        varlist = r_str.split()
        list_addr.append(base_addr)
        list_addr.append(base_addr + width)
        list_feature.append(str2hex(varlist[0]))
        list_feature.append(str2hex(varlist[1]))
        base_addr = base_addr + rel_addr
        r_str = fp.readline()
    fp.close()

    return list_addr, list_feature	

In [2]:
from pynq import Overlay
overlay = Overlay('./../bitcl/tensorcore.bit')

In [3]:
overlay.is_loaded()

True

In [4]:
tensor_stat = overlay.tensorcore_0

In [5]:
tensor_stat.write(0x00,0b0000_00_0_0001_0000_0_0001_00000000000_1)

In [6]:
overlay.tensorcore_0.weight('weight.txt')

In [7]:
overlay.tensorcore_0.feature('feature.txt')

In [8]:
overlay.tensorcore_0.bias('bias.txt')

In [9]:
psum0 = tensor_stat.read(0x80)
psum1 = tensor_stat.read(0x84)
print("The tensor results are",hex(psum1),hex(psum0)) 

The tensor results are 0x352a1c1b 0x1a0c0b1e


In [10]:
overlay.tensorcore_0.cfg_bias(0)

In [11]:
config = tensor_stat.read(0x00)
print("The confid is",bin(config)) 

The confid is 0b1000000001000000000001


In [12]:
overlay.tensorcore_0.cfg_featsel(2,1)

In [13]:
config = tensor_stat.read(0x00)
print("The confid is",bin(config)) 

The confid is 0b100100001000000001000000000001


In [14]:
overlay.tensorcore_0.cfg_wgtsel(2)

In [15]:
config = tensor_stat.read(0x00)
print("The confid is",bin(config)) 

The confid is 0b100100001000000001000000000101


In [19]:
overlay.tensorcore_0.conv3x3(4)

In [9]:
tensor_stat.write(0x04,0x04030201)
tensor_stat.write(0x08,0x08070605)
tensor_stat.write(0x0c,0x0c0b0a09)
tensor_stat.write(0x10,0x14131211)
tensor_stat.write(0x14,0x18171615)
tensor_stat.write(0x18,0x1c1b1a19)
tensor_stat.write(0x1c,0x24232221)
tensor_stat.write(0x20,0x28272625)
tensor_stat.write(0x24,0x2c2b2a29)
tensor_stat.write(0x28,0x34333231)
tensor_stat.write(0x2c,0x38373635)
tensor_stat.write(0x30,0x3c3b3a39)
tensor_stat.write(0x34,0x44434241)
tensor_stat.write(0x38,0x48474645)
tensor_stat.write(0x3c,0x4c4b4a49)
tensor_stat.write(0x40,0x54535251)
tensor_stat.write(0x44,0x58575655)
tensor_stat.write(0x48,0x5c5b5a59)

In [10]:
tensor_stat.write(0x104,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x100,    0b0000_0000000_0000000_0000000_0000001)
tensor_stat.write(0x124,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x120,    0b0000_0000000_0000000_0000001_0000000)
tensor_stat.write(0x144,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x140,    0b0000_0000000_0000001_0000000_0000000)
tensor_stat.write(0x164,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x160,    0b0000_0000001_0000000_0000000_0000000)
tensor_stat.write(0x184,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x180,    0b0001_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1a4,0b0_0000000_0000000_0000000_0000001_000)
tensor_stat.write(0x1a0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1c4,0b0_0000000_0000000_0000001_0000000_000)
tensor_stat.write(0x1c0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1e4,0b0_0000000_0000001_0000000_0000000_000)
tensor_stat.write(0x1e0,    0b0000_0000000_0000000_0000000_0000000)

In [16]:
tensor_stat.write(0x10c,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x108,    0b0000_0000000_0000000_0000001_0000001)
tensor_stat.write(0x12c,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x128,    0b0000_0000000_0000000_0000001_0000000)
tensor_stat.write(0x14c,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x148,    0b0000_0000000_0000001_0000000_0000000)
tensor_stat.write(0x16c,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x168,    0b0000_0000001_0000000_0000000_0000000)
tensor_stat.write(0x18c,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x188,    0b0001_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1ac,0b0_0000000_0000000_0000000_0000001_000)
tensor_stat.write(0x1a8,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1cc,0b0_0000000_0000000_0000001_0000000_000)
tensor_stat.write(0x1c8,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1ec,0b0_0000000_0000001_0000000_0000000_000)
tensor_stat.write(0x1e8,    0b0000_0000000_0000000_0000000_0000000)

In [17]:
tensor_stat.write(0x114,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x110,    0b0000_0000000_0000000_0000000_0000001)
tensor_stat.write(0x134,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x130,    0b0000_0000000_0000000_0000001_0000000)
tensor_stat.write(0x154,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x150,    0b0000_0000000_0000001_0000000_0000000)
tensor_stat.write(0x174,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x170,    0b0000_0000001_0000000_0000000_0000000)
tensor_stat.write(0x194,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x190,    0b0001_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1b4,0b0_0000000_0000000_0000000_0000001_000)
tensor_stat.write(0x1b0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1d4,0b0_0000000_0000000_0000001_0000000_000)
tensor_stat.write(0x1d0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1f4,0b0_0000000_0000001_0000000_0000000_000)
tensor_stat.write(0x1f0,    0b0000_0000000_0000000_0000000_0000000)

In [18]:
tensor_stat.write(0x114,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x110,    0b0000_0000000_0000000_0000000_0000001)
tensor_stat.write(0x134,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x130,    0b0000_0000000_0000000_0000001_0000000)
tensor_stat.write(0x154,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x150,    0b0000_0000000_0000001_0000000_0000000)
tensor_stat.write(0x174,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x170,    0b0000_0000001_0000000_0000000_0000000)
tensor_stat.write(0x194,0b0_0000000_0000000_0000000_0000000_000)
tensor_stat.write(0x190,    0b0001_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1b4,0b0_0000000_0000000_0000000_0000001_000)
tensor_stat.write(0x1b0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1d4,0b0_0000000_0000000_0000001_0000000_000)
tensor_stat.write(0x1d0,    0b0000_0000000_0000000_0000000_0000000)
tensor_stat.write(0x1f4,0b0_0000000_0000001_0000000_0000000_000)
tensor_stat.write(0x1f0,    0b0000_0000000_0000000_0000000_0000000)